In [19]:
import pandas as pd

import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets
from datetime import date, timedelta

In [163]:
url = 'https://raw.githubusercontent.com/tonmcg/US_County_Level_Election_Results_08-16/master/2016_US_County_Level_Presidential_Results.csv'

In [164]:
df = pd.read_csv(url, error_bad_lines=False)

In [165]:
utah = df[df.state_abbr == 'UT'].iloc[:,1:10].reset_index(drop=True)
#add the year of the election
utah['year'] = 2016
utah = utah[['county_name','total_votes','votes_dem','votes_gop','year']]
utah['county_name'] = utah['county_name'].str.split('County').str[0]

In [166]:
# NY Times
utah2020 = pd.read_html('https://www.nytimes.com/interactive/2020/11/03/us/elections/results-utah.html')[1]
utah2020 = utah2020[['County','Margin','Est. votes reported','Total votes','Absentee']].iloc[:-1,:]
utah2020['year'] = 2020
utah2020['margin'] = utah2020['Margin'].str.split('+').str[1].fillna(0).astype(int)
utah2020['direction'] = utah2020['Margin'].str.split('+').str[0].fillna(0)
utah2020['votes_one'] = utah2020['Total votes'].astype(int) * (0.5 +  (utah2020['margin'] / 100 / 2))
utah2020['votes_two'] = utah2020['Total votes'].astype(int) * (0.5 -  (utah2020['margin'] / 100 / 2))
utah2020 = utah2020.sort_values(by='direction')
utah2020['votes_dem'] = 'other'
utah2020['votes_gop'] = 'other'
utah2020.loc[utah2020['direction'] == 'Biden ', 'votes_dem'] = utah2020['votes_one']
utah2020.loc[utah2020['direction'] == 'Trump ', 'votes_dem'] = utah2020['votes_two']
utah2020.loc[utah2020['direction'] == 'Biden ', 'votes_gop'] = utah2020['votes_two']
utah2020.loc[utah2020['direction'] == 'Trump ', 'votes_gop'] = utah2020['votes_one']
utah2020 = utah2020[['County','Total votes','votes_dem','votes_gop','year','Est. votes reported']]
utah2020.columns = ['county_name','total_votes','votes_dem','votes_gop','year','reported']

In [169]:
df = pd.concat([utah,utah2020], sort=False)
df['reported'] = df.reported.fillna('100%')
df

,county_name,total_votes,votes_dem,votes_gop,year,reported
0,Beaver,2490,264,1838,2016,100%
1,Box Elder,16548,1814,10324,2016,100%
2,Cache,35879,6705,16643,2016,100%
3,Carbon,7818,1696,5182,2016,100%
4,Daggett,465,76,327,2016,100%
5,Davis,104661,22575,46985,2016,100%
6,Duchesne,6932,500,5505,2016,100%
7,Emery,4263,379,3402,2016,100%
8,Garfield,2294,352,1577,2016,100%
9,Grand,4456,1932,1934,2016,100%


In [172]:
pycred = pygsheets.authorize(service_file='/Users/paulbrown/Documents/credentials.json')
#opening the gsheet and sheet you want to work with
ss = pycred.open('Utah Election Results')[0]
#overwrite what is in the sheet with your df
ss.set_dataframe(df,(1,1))